## Collect a Company's Patent Porfolio

This notebook will attempt to amass, for any given company, their entire worldwide patent porfolio.\*

### U.S. Portfolio

Collecting their U.S. portfolio will proceed in a few steps:

1. Collect all applications ever assigned to the company

    a. Those applications naming the company as an applicant
    b. Those assigned to the company via an assignment
    
2. Prune all applications assigned out of the company, as recorded in USPTO assignment records

\*NOTE: I say "attempt" because the notebook relies on the USPTO system, and the International Patent Document (INPADOC) service. Not all foreign jurisdictions participate in INPADOC, and not all INPADOC member states are as good at contributing to the system as we would like. So coverage is good, but not perfect.

First, we do our standard imports, and grab the USApplication and Assignment objects from patent_client

In [1]:
import pandas as pd
from patent_client import USApplication, Assignment

company_name = 'Tesla Motors'

### Step 1.a: Collect all applications naming the company as applicant

In [2]:
applicant_apps = USApplication.objects.filter(first_named_applicant=company_name).values_list('appl_id', flat=True)

### Step 1.b: Collect all applications assigned to the company

In [7]:
# The assigned apps is either a single value, or a list of values if more than one property was assigned
assigned_apps = list()
for a in Assignment.objects.filter(assignee=company_name):
    for p in a.properties:
        assigned_apps.append(p.appl_id)

And now we combine the two

In [8]:
all_apps = list(applicant_apps) + assigned_apps
all_apps = list(set(all_apps)) # use set to deduplicate the list

print(f'Total U.S. Applications Ever Owned by {company_name}: {len(all_apps)}')

Total U.S. Applications Ever Owned by Tesla Motors: 243


### Step 2: Collect all applications ever assigned out of the company

In [9]:
assignments_out = Assignment.objects.filter(assignor=company_name)
assigned_out_apps = list()

for assignment in assignments_out:
    if 'assignors interest' in assignment.conveyance_text.lower():
        a = assignment.appl_num
        if isinstance(a, list):
            assigned_out_apps += a
        else:
            assigned_out_apps.append(a)
            
len(assigned_out_apps)

4

### Step 3: Subtract one set from the other

In [12]:
owned_apps = list(set(all_apps) - set(assigned_out_apps))
len(owned_apps)


243

### Step 4: Generate status report

In [13]:
import pandas as pd
from patent_client.parser import parse

apps = USApplication.objects.filter(appl_id=owned_apps)

df = pd.DataFrame.from_records(apps.values(
    'appl_id', 
    'app_filing_date', 
    'app_early_pub_number',
    'app_early_pub_date',
    'patent_number',
    'patent_issue_date',
    'patent_title',
    'inventors__0__name',
    'applicants__0__name',
    'app_status',
    'app_status_date'
))

def display_num(number):
    if number:
        return parse(number).display()
    else:
        return number

#df['patent_title'] = df['patent_title'].apply(str.capitalize)
#df['appl_id'] = df['appl_id'].apply(display_num)
#df['app_early_pub_number'] = df['app_early_pub_number'].apply(display_num)
#df['patent_number'] = df['patent_number'].apply(display_num)
us_df = df
us_df

,appl_id,app_filing_date,app_early_pub_number,app_early_pub_date,patent_number,patent_issue_date,patent_title,inventors__0__name,applicants__0__name,app_status,app_status_date
0,PCT/US06/18529,2006-05-12,WO/2006/0124663,2006-11-23,None,None,"METHOD AND APPARATUS FOR MOUNTING, COOLING, CO...",JEFFREY STRAUBEL,None,RO PROCESSING COMPLETED-PLACED IN STORAGE,2018-11-15
1,PCT/US07/03986,2007-02-13,WO/2007/0095327,2007-08-23,None,None,SYSTEM AND METHOD FOR FUSIBLY LINKING BATTERIES,DAVID LYONS,None,RO PROCESSING COMPLETED-PLACED IN STORAGE,2018-11-14
2,PCT/US07/10289,2007-04-27,WO/2007/0127397,2007-11-08,None,None,SYSTEM AND METHOD FOR INTERCONNECTION OF BATTE...,DORIAN WEST,None,RO PROCESSING COMPLETED-PLACED IN STORAGE,2018-11-14
3,PCT/US07/10799,2007-05-02,WO/2007/0145726,2007-12-21,None,None,SYSTEM AND METHOD FOR AN EFFICIENT ROTOR FOR A...,DAVID LYONS,None,RO PROCESSING COMPLETED-PLACED IN STORAGE,2018-11-14
4,PCT/US07/12841,2007-05-30,WO/2007/0143033,2007-12-13,None,None,SYSTEM AND METHOD FOR INHIBITING THE PROPAGATI...,JEFFREY STRAUBEL,None,RO PROCESSING COMPLETED-PLACED IN STORAGE,2018-11-14
5,13621943,2012-09-18,US20130076076A1,2013-03-28,8807637,2014-08-19,ANGLED FRONT HOOD SEALING ASSEMBLY,Matthew R. Partsch,None,Patented Case,2014-07-30
6,13624900,2012-09-22,US20130015823A1,2013-01-17,9209631,2015-12-08,CHARGE RATE MODULATION OF METAL-AIR CELLS AS A...,Weston Arthur Hermann,None,Patented Case,2015-11-18
7,13626864,2012-09-25,US20130076047A1,2013-03-28,9103143,2015-08-11,DOOR HANDLE APPARATUS FOR VEHICLES,David Wheeler,None,Patented Case,2015-07-22
8,13627903,2012-09-26,US20130079984A1,2013-03-28,9151089,2015-10-06,CONTROLLER APPARATUS AND SENSORS FOR A VEHICLE...,Joris Aerts,None,Patented Case,2015-09-16
9,13628034,2012-09-26,US20140022811A1,2014-01-23,8807807,2014-08-19,ILLUMINATION APPARATUS FOR VEHICLES,David Wheeler,None,Patented Case,2014-07-30


### Foreign Patent Portfolio

Now we will fetch the company's wordwide portfolio, as it appears in the Inpadoc system maintained by the EPO

In [14]:
from patent_client_plus import Inpadoc

foreign = Inpadoc.objects.filter(applicant=company_name)

And now, a status report:

In [15]:
foreign_df = pd.DataFrame.from_records(foreign.values('country', 'application', 'filing_date', 'publication', 'publication_date', 'title', 'applicants__0', 'inventors__0', 'legal__-1__description', 'legal__-1__date'))
foreign_df = foreign_df[foreign_df['country'] != 'US']
foreign_df['title'] = foreign_df['title'].apply(str.capitalize)
foreign_df['status'] = foreign_df['legal_-1_description']
foreign_df['status_date'] = foreign_df['legal_-1_date']
foreign_df = foreign_df.drop(labels=['legal_-1_description', 'legal_-1_date'], axis=1)
foreign_df

KeyError: 'legal_-1_description'

And now we will take our results, and write to an excel file

In [ ]:
writer = pd.ExcelWriter(f'{company_name}_portfolio.xlsx')
us_df.to_excel(writer, sheet_name='US Portfolio')
foreign_df.to_excel(writer, sheet_name='Foreign Portfolio')
writer.save()